In [14]:
import cv2
from matplotlib import pyplot as plt
import os
from os import path
import re
import numpy as np
import shutil

In [2]:
NEW_DIR_COLLECTIONS = [
    '/media/yeeef/Seagate Expansion Drive/training_data/selected_images_1_camera_polygon',
    '/media/yeeef/Seagate Expansion Drive/training_data/selected_images_2_camera_polygon',
    '/media/yeeef/Seagate Expansion Drive/training_data/selected_images_3_camera_polygon',
    '/media/yeeef/Seagate Expansion Drive/training_data/selected_images_4_camera_polygon',
    '/media/yeeef/Seagate Expansion Drive/training_data/selected_images_5_camera_polygon',
    '/media/yeeef/Seagate Expansion Drive/training_data/selected_images_6_camera_polygon',
    '/media/yeeef/Seagate Expansion Drive/training_data/selected_images_8_camera_polygon',

]
BASE_DIR = NEW_DIR_COLLECTIONS[0]
LOG_PATTERN = re.compile(r'(\d+).log')


In [3]:
group_dirs = list(os.scandir(BASE_DIR))

In [4]:
demo_group_dir = group_dirs[0]
log_dir = path.join(demo_group_dir.path, 'image', 'color_depth_log')
demo_files = list(os.scandir(log_dir))
img_files = [file for file in demo_files if path.splitext(file.name)[1] in ['.png', '.jpg']]
label_log_files = [file for file in demo_files if path.splitext(file.name)[1] == '.log' and file.name != 'camera_params.log']

In [5]:
label_log_files

[<DirEntry '0.log'>,
 <DirEntry '1.log'>,
 <DirEntry '2.log'>,
 <DirEntry '3.log'>,
 <DirEntry '4.log'>]

In [6]:
class LogFile(object):
    """ log parser, come from plane_estimate.py """
    def __init__(self, log_id, content_lines):
        self.log_id = log_id
        self.content_lines = content_lines
        self.num_lines = len(content_lines)
        self.num_polygon = 0
        self.polygons = []
        self._file_ptr = 0

    def _ptr_advance(self):
        self._file_ptr += 1

    def _ptr_advance_by(self, step):
        self._file_ptr += step

    def _is_eof(self):
        if self._file_ptr > self.num_lines:
            return True
        else:
            return False

    def _ptr_line_content(self):
        if not self._is_eof():
            return self.content_lines[self._file_ptr]
        else:
            raise IOError('{} log, ptr {} exceeds eof!'.format(self.log_id, self._file_ptr))

    def _parse_polygon_meta(self, meta_str):
        fields = meta_str.split(' ')
        polygon_id = fields[0].strip()
        num_points = int(fields[1].strip())
        return polygon_id, num_points

    def _parse_polygon_info(self, num_points):
        point_list = []
        for i in range(num_points):
            self._ptr_advance()
            line = self._ptr_line_content()
            fields = line.split(' ')
            x = round(float(fields[0].lower()))
            y = round(float(fields[1].lower()))
            point_list.append((x, y))

        return point_list

    def parse_content(self):
        meta = self.content_lines[0]
        self.num_polygon = int(meta.split(' ')[-1].strip())
        polygon_list = []
        try:
            for i in range(self.num_polygon):
                self._ptr_advance()
                polygon_meta = self._ptr_line_content()
                _, num_points = self._parse_polygon_meta(polygon_meta)
                point_list = self._parse_polygon_info(num_points)
                if len(point_list) > 2:
                    polygon_list.append(point_list)
        except IOError as ioe:
            print('Error: ', ioe)
            exit()

        return polygon_list

In [ ]:
for label_log in label_log_files[:1]:
    match = re.search(LOG_PATTERN, label_log.name)
    log_id = int(match.group(1))
    content_lines = open(label_log.path).readlines()
    log_parser = LogFile(log_id, content_lines)
    polygon_list = log_parser.parse_content()
    img_path = path.join(path.dirname(label_log.path), '{}.png'.format(log_id))
    print(img_path)
    img = cv2.imread(img_path)
    label = np.zeros([img.shape[0], img.shape[1]])
    cv2.fillConvexPoly(img, polygon_list[0], (255, 255, 255))
    print(img)
    print(polygon_list)

/media/yeeef/Seagate Expansion Drive/training_data/selected_images_1_camera_polygon/29/image/color_depth_log/0.png


In [10]:
os.listdir(log_dir)

['0.exr',
 '0.log',
 '0.png',
 '1.exr',
 '1.log',
 '1.png',
 '2.exr',
 '2.log',
 '2.png',
 '3.exr',
 '3.log',
 '3.png',
 '4.exr',
 '4.log',
 '4.png',
 'camera_params.log',
 'check_episolar',
 'controlPoints.csv']

In [7]:
os.listdir(demo_group_dir.path)

['crmeta.db',
 'image',
 'IMG_8279.JPG',
 'IMG_8280.JPG',
 'IMG_8281.JPG',
 'IMG_8282.JPG',
 'IMG_8283.JPG',
 'RealCapture']

In [20]:
## rename mvsnet output dir

dataset_dir = '/data3/lyf/MVSNET/mvsnet_test/standard_dataset/illumination_part1_adaptives'
result_dir = '/data3/lyf/MVSNET/mvsnet_test/results/illumination_part1_adaptives'
new_result_dir = '/data3/lyf/MVSNET/mvsnet_test/results/new_illumination_part1_adaptives'

In [21]:
dataset_groups = sorted(os.listdir(dataset_dir), key=int)
result_groups = os.listdir(result_dir)
result_groups = [group for group in result_groups if 'log' not in group]
result_groups = sorted(result_groups, key=int)

In [22]:
assert len(result_groups) == len(dataset_groups)

In [23]:
idx_mapping = dict(zip(result_groups, dataset_groups))

In [24]:
print(idx_mapping)
for group in result_groups:
    if group in idx_mapping:
        shutil.move(path.join(result_dir, group), path.join(new_result_dir, idx_mapping[group]))
        
shutil.rmtree(result_dir)
shutil.move(new_result_dir, result_dir)

{'0': '1', '1': '2', '2': '4', '3': '5', '4': '6', '5': '7', '6': '8', '7': '9', '8': '12', '9': '13', '10': '14', '11': '17', '12': '19', '13': '20', '14': '22', '15': '24', '16': '25', '17': '26', '18': '27', '19': '28', '20': '29', '21': '31', '22': '32', '23': '33', '24': '34', '25': '36', '26': '39', '27': '42', '28': '43', '29': '44', '30': '45', '31': '46', '32': '47', '33': '48', '34': '49', '35': '50', '36': '51', '37': '52', '38': '53', '39': '54', '40': '55', '41': '56', '42': '57', '43': '58', '44': '59', '45': '60', '46': '64', '47': '65', '48': '66', '49': '68', '50': '69', '51': '70', '52': '73', '53': '74', '54': '76', '55': '77', '56': '78', '57': '79', '58': '81', '59': '82', '60': '85', '61': '86', '62': '88', '63': '89', '64': '90', '65': '91', '66': '93', '67': '94', '68': '95', '69': '96', '70': '97', '71': '99', '72': '100'}
